In [ ]:
import numpy as np
import os
import pandas as pd
from utils import save_tif , read_tif, include_file_extension
from config import paths
from preprocessing import preprocess_images
from denoising import train_n2v, denoise_images
from blobDetection import convert_blobs_df
from cropping import blobs_to_crops

# Load, preprocess and denoise images

In [ ]:
# Select image filename - it should correspond to a file in DATA_DIR/Images/Raw
filename = "20_mix_images" #recommended without the .tif but it should work with that as well

In [ ]:
# Preprocess the image stack
path_to_raw_images = os.path.join(paths["raw"],filename)
preprocessed_images, preprocessing_kept_inds = preprocess_images(read_tif(path_to_raw_images))

# Save the preprocessed images and also a csv connecting the new indices to the original ones in the images
path_to_preprocessed_images = os.path.join(paths["preprocessed"],filename)
save_tif(path_to_preprocessed_images,preprocessed_images)
pd.DataFrame({"inds_raw":preprocessing_kept_inds[0],"inds_preprocessed":preprocessing_kept_inds[1]}).reset_index(drop=True).to_csv(os.path.join(paths["preprocessed"],include_file_extension(filename,"csv")))

### Warning: N2V training is slow, do it with GPU and lots of RAM.
### If you already have a trained model, skip the next cell

In [ ]:
# Select model name
n2v_model_name = "my_n2v_model"
# Train_epochs should be >=10 for serious models
# N_images_for_train should be >=100 or None for using all images in the stack to train
# random_seed controls how the data is shuffled for training
train_n2v(filename, n2v_model_name, train_epochs=1, N_images_for_train=1, random_seed=0)

In [ ]:
n2v_model_name = "mir_model_v3"
denoised_images = denoise_images(read_tif(os.path.join(paths["preprocessed"],filename)),n2v_model_name)
save_tif(os.path.join(paths["denoised"],filename),denoised_images)

# Identify blobs in images

### Before running this cell, run ThunderSTORM on the denoised images stack and save the output in the blobs directory

In [ ]:
name_of_thunderstorm_output = os.path.join(paths['blobs'],"my_thunderstorm_output.csv")
# Generate the blobs dataframe (and save it in the Blobs folder)
_=convert_blobs_df(name_of_thunderstorm_output, filename, save_files=True)

### Convert the blobs to crops for ML and for V_TIMDER
#### Consider providing a smaller image file or blobs_df for the V_TIMDER export

In [ ]:
# Convert the blobs dataframe obtained previously to crops
crop_dimensions = np.array([[-6, 18], [-5, 5]]) # [[-up,down],[-left,right]] with respect to the blob

blobs_df = pd.read_csv(os.path.join(paths["blobs"],include_file_extension(filename,"tif")[:-4]+".csv"))

preprocessed_images = read_tif(os.path.join(paths["preprocessed"],filename))
keep_inds,crops,crops_L,crops_XL = blobs_to_crops(preprocessed_images,blobs_df,crop_dimensions)

#This exports to V_TIMDER
save_tif(os.path.join(paths["v_timder_noisy"],"Normal",filename),crops)
save_tif(os.path.join(paths["v_timder_noisy"],"Large",filename),crops_L)
save_tif(os.path.join(paths["v_timder_noisy"],"XL",filename),crops_XL)
blobs_df.loc[keep_inds].reset_index(drop=True).to_csv(os.path.join(paths["v_timder_tables"],include_file_extension(filename,"tif")[:-4]+".csv"))

denoised_images = read_tif(os.path.join(paths["denoised"],filename))
keep_inds,crops,crops_L,crops_XL = blobs_to_crops(denoised_images,blobs_df,crop_dimensions)

#This exports to V_TIMDER
save_tif(os.path.join(paths["v_timder_denoised"],"Normal",filename),crops)
save_tif(os.path.join(paths["v_timder_denoised"],"Large",filename),crops_L)
save_tif(os.path.join(paths["v_timder_denoised"],"XL",filename),crops_XL)

# This does not export to V_TIMDER
save_tif(os.path.join(paths["crops"],filename),crops)